Scrape forecast data from PDFs on government archive site

In [25]:

import pandas as pd
from tabula import read_pdf
import requests, bs4, time
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
from datetime import date
import pickle

def extract_links(soup):
    links=[]
    for div in page_soup.find_all(name='div', attrs={'id':'primaryContentFull'}):
        for el in div.find_all(name='a'):
            links.append(el['href'])
    return [l  for l in links if '.pdf' in l]

def extract_links_2(soup):
    links =[]
    for div in soup.find_all(name='h2', attrs={'class':'title'}):
        links.append(div.a['href'])
    return links

def get_links():
    links=[]
    url="https://webarchive.nationalarchives.gov.uk/20100407010846/http://www.hm-treasury.gov.uk/data_forecasts_index.htm"
    req=Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    a=extract_links(page_soup)
    for element in a:
        links.append(element)
    return links

def get_titles(links):
    titles=[]
    url="https://webarchive.nationalarchives.gov.uk/20100407010846/http://www.hm-treasury.gov.uk/data_forecasts_index.htm"
    req=Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage=urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    for div in page_soup.find_all(name='div', attrs={'id':'primaryContentFull'}):
        for i in links:
            for el in div.find_all(name='a',attrs={'href':i}):
                month = calendar.month_name[int(el.text.split('/')[1])]
                year = int(el.text.split('/')[2].split('(')[0])
                if year < 50:
                    year2=year+2000
                else:
                    year2=year+1900
            titles.append(month+'_'+str(year2))
    return titles






In [30]:
links_dict =dict(zip(get_titles(get_links()), get_links()))

In [41]:
[x for x in get_links() if x not in list(links_dict.values())]

# there were two publications of forecasts in jan 1998 

['/20100407010846/http://www.hm-treasury.gov.uk/d/293.pdf']

In [123]:
data_dict = {}

for k in links_dict:
    try:
        u = "https://webarchive.nationalarchives.gov.uk" + links_dict[k]
        test = read_pdf(u,pages=[1])
    except:
        u = "https://webarchive.nationalarchives.gov.uk/+/" + links_dict[k].split('20100407010846/')[1]
        
    if k in ['December_2009','November_2009']:
        first = 7
        second = 10
    elif k in ['March_2010','January_2010','October_1999','September_1999']:
        first = 6
        second = 9
    elif k in ['February_2010', 'December_2003','August_2003','February_2002','January_2002','November_1997','March_2000','February_2000', 'December_1999','May_1999','January_1998']:
        first = 5
        second = 8
    else:
        first = 4
        second = 7
    r1 = read_pdf(u,pages=[first])
    r2 = read_pdf(u,pages=[second])


    data_dict[k] = {first:r1,second:r2}


Got stderr: May 06, 2020 6:51:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 06, 2020 6:51:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 06, 2020 6:51:17 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 06, 2020 6:51:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 06, 2020 6:51:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
May 06, 2020 6:51:19 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: May 06, 2020 6:51:32 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
May 06, 2020 6:51:32 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
May 06, 2020 6:51:32 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
May 06, 2020 6:51:32 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
May 06, 2020 6:51:32 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
May 06, 2020 6:51:32 PM org.apache.pdfbox.pdmodel.font.PDType0Font toUnicode
May 06, 2020 6:51:32 PM org.apache.pdf

In [126]:
# this is a document on lobbying for green energy in Germany
data_dict.pop('March_2002', None)
# it can't read this
data_dict.pop('July_1999', None)

{4: [], 7: []}

In [132]:
for k in data_dict:
    print(k)
    for j in data_dict[k]:
        print(len(data_dict[k][j]))
        try:
            print('GDP: '+ str(data_dict[k][j][0].iloc[:,-1].str.contains('GDP').sum()))
        except:
            print('GDP: Error')

March_2010
1
GDP: 1
1
GDP: 1
February_2010
1
GDP: 1
1
GDP: 1
January_2010
1
GDP: 1
1
GDP: 1
December_2009
1
GDP: 1
1
GDP: 1
November_2009
1
GDP: 1
1
GDP: 1
October_2009
1
GDP: 1
1
GDP: 1
September_2009
1
GDP: 1
1
GDP: 1
August_2009
1
GDP: 1
1
GDP: 1
July_2009
1
GDP: 1
1
GDP: 1
June_2009
1
GDP: 1
1
GDP: 1
May_2009
1
GDP: 1
1
GDP: 1
April_2009
1
GDP: 1
1
GDP: 1
March_2009
1
GDP: 1
1
GDP: 1
February_2009
1
GDP: 1
1
GDP: 1
January_2009
1
GDP: 1
1
GDP: 1
December_2008
1
GDP: 1
1
GDP: 1
November_2008
1
GDP: 1
1
GDP: 1
October_2008
1
GDP: 1
1
GDP: 1
September_2008
1
GDP: 1
1
GDP: 1
August_2008
1
GDP: 1
1
GDP: 1
July_2008
1
GDP: 1
1
GDP: 1
June_2008
1
GDP: 1
1
GDP: 1
May_2008
1
GDP: 1
1
GDP: 1
April_2008
1
GDP: 1
1
GDP: 1
March_2008
1
GDP: 1
1
GDP: 1
February_2008
1
GDP: 1
1
GDP: 1
January_2008
1
GDP: 1
1
GDP: 1
December_2007
1
GDP: 1
1
GDP: 1
November_2007
1
GDP: 1
1
GDP: 1
October_2007
1
GDP: 1
1
GDP: 1
September_2007
1
GDP: 1
1
GDP: 1
August_2007
1
GDP: 1
1
GDP: 1
July_2007
1
GDP: 1
1
GDP: 

In [133]:
with open('C:/Users/lundr/economics/GDP_Forecasts/raw_data/all_data_archive.pkl','wb') as f:
    pickle.dump(data_dict,f)